(Introduce the project)
(Introduce the dataset)

## Gather

(import statements)

In [1]:
import json
import numpy as np
import pandas as pd
import requests
import tweepy

ModuleNotFoundError: No module named 'tweepy'

(load twitter_archive_enhanced.csv)

(load image_predictions.tsv)

(use twitter api for json data)
(test on small sample, once working get complete data set once and then comment out)
(temp: load from a saved local file)

## Assess

(Explore each using .info)

#### Quality
##### `twitter_archive_enhanced` table
- x 
- x
- x

##### `image_predictions` table
- x
- x

##### `additional_data` table
- x
- x
- x

#### Tidiness
- x
- x

## Clean

(make copies of datasources)

### (Problem 1)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 2)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 3)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 4)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 5)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 6)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 7)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 8)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 9)

(Problem: ...)

##### Define
x

##### Code

##### Test

### (Problem 10)

(Problem: ...)

##### Define
x

##### Code

##### Test

## Storage

(store each table as a csv file with main file called tiwtter_archive_master.csv)

## Analysis

(make at least 3 insights)